In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import os



In [4]:
train_images_path = r"C:\Users\ASUS\Desktop\INT422\images\barbaracle.png"
IMAGE_WIDTH = 60
IMAGE_HEIGHT = 160
size = (IMAGE_HEIGHT, IMAGE_WIDTH)
batch_size = 32
epochs = 2

In [12]:
print(os.path.abspath("C:/Users/ASUS/Desktop/INT422/pokemon.csv"))

/content/C:/Users/ASUS/Desktop/INT422/pokemon.csv


In [17]:
class CustomDataset(Dataset):
    def __init__(self, data, path, transform=None):
        self.data = data
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img1 = Image.open(self.path + self.data["image1"][idx])
        img2 = Image.open(self.path + self.data["image2"][idx])
        label = self.data["label"][idx]

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, label

In [18]:
train_data = pd.read_csv("./pokemon.csv")
train_dataset = CustomDataset(train_data, train_images_path, transform=transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor()
]))

In [19]:
train_set, test_set = train_test_split(train_dataset, test_size=0.2, random_state=42)



KeyError: 'image1'

In [ ]:
train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size)



In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(True)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2, output_padding=1, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, 3, stride=2, output_padding=1, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 3, 3, stride=2, output_padding=1, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.decoder(x)
        return x

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
         def forward(self, img1, img2):

In [ ]:
model = Autoencoder()
if torch.cuda.is_available():
    model = model.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train():
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for batch_idx, (img1, img2, label) in enumerate(train_loader):
            if torch.cuda.is_available():
                img1, img2, label = img1.cuda(), img2.cuda(), label.cuda()

            optimizer.zero_grad()
            outputs = model(img1, img2)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()

            if batch_idx % 50 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {(100*correct/total):.2f}%")


In [ ]:
def test():
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for img1, img2, label in test_loader:
            if torch.cuda.is_available():
                img1, img2, label = img1.cuda(), img2.cuda(), label.cuda()

            outputs = model(img1)
            loss = criterion(outputs, img1)  # Reconstruction loss
            running_loss += loss.item()

    print(f"Test Loss: {running_loss/len(test_loader):.4f}")


In [ ]:
train()


In [ ]:
test()
